In [1]:
#hopeのページからe-Learningの小テストの評点csvと目標設定のcsvをスクレイピングしてダウンロードするプログラム
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
from dotenv import load_dotenv
from datetime import datetime
from zoneinfo import ZoneInfo
import shutil

load_dotenv()

# 環境変数
USERNAME = os.getenv("USERNAME")
PASSWORD = os.getenv("PASSWORD")
SELENIUM_HOST = os.getenv("SELENIUM_HOST", "selenium")

# フォルダパス
SELENIUM_DOWNLOAD_DIR = "/home/seluser/Downloads" #Downloadsフォルダにseluserの書き込み権限がないので出力できない->した
JUPYTER_SHARED_DIR = "/home/jovyan/work/shared_downloads"

jst_now = datetime.now(ZoneInfo("Asia/Tokyo"))
TODAY_STR = jst_now.strftime("%Y%m%d")

TEST_TARGET_FOLDER = os.path.join(JUPYTER_SHARED_DIR, "MiniTest", TODAY_STR)
PURPOSE_TARGET_FOLDER = os.path.join(JUPYTER_SHARED_DIR, "Purpose", TODAY_STR)
COMMENT_TARGET_FOLDER = os.path.join(JUPYTER_SHARED_DIR, "Comment", TODAY_STR)

# 小テストCSVのURL一覧
TEST_CSV_URLS = [
    "https://hope.fun.ac.jp/grade/export/txt/index.php?id=2059", #グループ1（keitest4: 全支援あり）
    "https://hope.fun.ac.jp/grade/export/txt/index.php?id=2409", #グループ2（keilabtest5: 支援なし）
    "https://hope.fun.ac.jp/grade/export/txt/index.php?id=2412", #グループ3（keilabtest7: 自律性のみ）
]

# 目標設定のcsvをダウンロードするリンク先の一覧配列
PURPOSE_CSV_URLS = {
 'Linux': {
     'keitest4': [
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130429&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130438&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130447&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130456&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130465&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130474&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130483&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130492&mode=overview'
     ],
     'keilabtest7': [
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139485&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139494&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139503&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139512&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139521&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139530&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139539&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139548&mode=overview'
     ]
 },
 'ネットワーク': {
     'keitest4': [
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130430&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130439&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130448&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130457&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130466&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130475&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130484&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130493&mode=overview'
     ],
     'keilabtest7': [
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139486&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139495&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139504&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139513&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139522&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139531&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139540&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139549&mode=overview'
     ]
 },
 'クラウド': {
     'keitest4': [
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130431&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130440&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130449&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130458&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130467&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130476&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130485&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130494&mode=overview'
     ],
     'keilabtest7': [
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139487&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139496&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139505&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139514&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139523&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139532&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139541&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139550&mode=overview'
     ]
 },
 'Java': {
     'keitest4': [
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130432&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130441&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130450&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130459&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130468&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130477&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130486&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130495&mode=overview'
     ],
     'keilabtest7': [
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139488&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139497&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139506&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139515&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139524&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139533&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139542&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139551&mode=overview'
     ]
 },
 'アルゴリズム': {
     'keitest4': [
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130433&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130442&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130451&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130460&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130469&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130478&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130487&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130496&mode=overview'
     ],
     'keilabtest7': [
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139489&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139498&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139507&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139516&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139525&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139534&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139543&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139552&mode=overview'
     ]
 },         
 'UML': {
     'keitest4': [
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130434&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130443&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130452&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130461&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130470&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130479&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130488&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130497&mode=overview'
     ],
     'keilabtest7': [
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139490&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139499&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139508&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139517&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139526&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139535&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139544&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139553&mode=overview'
     ]
 }, 
 'システム': {
     'keitest4': [
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130435&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130444&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130453&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130462&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130471&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130480&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130489&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130498&mode=overview'
     ],
     'keilabtest7': [
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139491&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139500&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139509&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139518&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139527&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139536&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139545&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139554&mode=overview'
     ]
 },  
 'マネジメント': {
     'keitest4': [
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130436&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130445&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130454&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130463&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130472&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130481&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130490&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130499&mode=overview'
     ],
     'keilabtest7': [
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139492&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139501&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139510&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139519&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139528&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139537&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139546&mode=overview',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139555&mode=overview'
     ]
 },           
}






# 目標設定のコメント内容のcsvをダウンロードするリンク先の一覧配列
COMMENT_PURPOSE_CSV_URLS = {
 'Linux': {
     'keitest4': [
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130429&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130438&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130447&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130456&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130465&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130474&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130483&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130492&mode=responses'
     ],
     'keilabtest7': [
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139485&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139494&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139503&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139512&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139521&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139530&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139539&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139548&mode=responses'
     ]
 },
 'ネットワーク': {
     'keitest4': [
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130430&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130439&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130448&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130457&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130466&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130475&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130484&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130493&mode=responses'
     ],
     'keilabtest7': [
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139486&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139495&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139504&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139513&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139522&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139531&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139540&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139549&mode=responses'
     ]
 },
 'クラウド': {
     'keitest4': [
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130431&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130440&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130449&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130458&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130467&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130476&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130485&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130494&mode=responses'
     ],
     'keilabtest7': [
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139487&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139496&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139505&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139514&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139523&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139532&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139541&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139550&mode=responses'
     ]
 },
 'Java': {
     'keitest4': [
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130432&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130441&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130450&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130459&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130468&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130477&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130486&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130495&mode=responses'
     ],
     'keilabtest7': [
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139488&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139497&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139506&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139515&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139524&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139533&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139542&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139551&mode=responses'
     ]
 },
 'アルゴリズム': {
     'keitest4': [
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130433&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130442&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130451&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130460&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130469&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130478&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130487&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130496&mode=responses'
     ],
     'keilabtest7': [
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139489&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139498&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139507&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139516&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139525&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139534&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139543&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139552&mode=responses'
     ]
 },         
 'UML': {
     'keitest4': [
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130434&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130443&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130452&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130461&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130470&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130479&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130488&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130497&mode=responses'
     ],
     'keilabtest7': [
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139490&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139499&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139508&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139517&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139526&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139535&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139544&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139553&mode=responses'
     ]
 }, 
 'システム': {
     'keitest4': [
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130435&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130444&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130453&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130462&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130471&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130480&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130489&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130498&mode=responses'
     ],
     'keilabtest7': [
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139491&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139500&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139509&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139518&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139527&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139536&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139545&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139554&mode=responses'
     ]
 },  
 'マネジメント': {
     'keitest4': [
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130436&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130445&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130454&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130463&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130472&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130481&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130490&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=130499&mode=responses'
     ],
     'keilabtest7': [
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139492&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139501&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139510&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139519&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139528&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139537&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139546&mode=responses',
         'https://hope.fun.ac.jp/mod/quiz/report.php?id=139555&mode=responses'
     ]
 },           
}




def setup_driver():
    options = Options()
    options.add_experimental_option("prefs", {
        "download.default_directory": SELENIUM_DOWNLOAD_DIR,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    })
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--headless=new")
    options.add_argument("--remote-debugging-port=9222")

    return webdriver.Remote(
        command_executor=f"http://{SELENIUM_HOST}:4444/wd/hub",
        options=options
    )

def login_to_hope(driver):
    login_start_url = "https://hope.fun.ac.jp/local/hope/login.php"
    driver.get(login_start_url)

    print("⌛ 「未来大の学生・教職員」リンクをクリック")
    driver.find_element(By.LINK_TEXT, "未来大の学生・教職員 FUN Students & Staff").click()

    print("⌛ ユーザー名・パスワード入力待機中")
    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.NAME, "username")))
    driver.find_element(By.NAME, "username").send_keys(USERNAME)
    driver.find_element(By.NAME, "password").send_keys(PASSWORD)
    driver.find_element(By.CSS_SELECTOR, "input[type='submit']").click()

    WebDriverWait(driver, 15).until(EC.url_changes(login_start_url))
    print("✅ ログイン成功")


def download_csv(driver, url, element):
    print(f"🌐 アクセス中: {url}")
    driver.get(url)
    download_button = None  # 初期化

    try:
        if element == 'id':
            download_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID, "id_submitbutton"))      
            )
        else:
            buttons = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, "btn-secondary"))
            )
            for button in buttons:
                if "ダウンロード" in button.text:
                    download_button = button
                    break  # 最初に見つけたものを使う（ここはハードコード）

        if download_button:
            time.sleep(2)
            download_button.click()
            print("⬇️ ダウンロード中...（3秒待機）")
            time.sleep(3)
        else:
            print("❌ ダウンロードボタンが見つかりませんでした（3秒待機）")
            time.sleep(3)

    except Exception as e:
        driver.get_screenshot_as_file("error.png")
        print(f"❌ ダウンロード失敗またはエラーが発生しました: {e}")    


#小テストcsv
def minitest_download_csv_from_urls(driver, urls, element):
    for i, url in enumerate(urls):
        download_csv(driver, url, element)


def minitest_move_downloaded_files():
    #/home/jovyan/work/shared_downloads/MiniTest/[今日の日付]/ のフォルダをなかったら作成、あってもif文抜けるだけ
    if not os.path.exists(TEST_TARGET_FOLDER):
        os.makedirs(TEST_TARGET_FOLDER)

    print("📂 ファイル移動開始...")
    for filename in os.listdir(JUPYTER_SHARED_DIR):
        file_path = os.path.join(JUPYTER_SHARED_DIR, filename)
        if os.path.isfile(file_path):
            target_path = os.path.join(TEST_TARGET_FOLDER, filename)
            shutil.move(file_path, target_path)
            print(f"✅ {filename} を {TEST_TARGET_FOLDER} に移動")



# 目標設定csv
def comment_purpose_download_csv_from_urls(driver, url_dict):
    count = 0
    for subject, courses in url_dict.items():
        for course_name, urls in courses.items():
            for i, url in enumerate(urls, 1):  # 1から始まるインデックス
                print(f"科目:{subject} コース名:{course_name} の {i} 番目のダウンロード")
                download_csv(driver, url, '')
                count += 1
                if count % 8 == 0:
                    print("💤 8件ごとに60秒休憩中...")
                    time.sleep(60)


def comment_purpose_move_downloaded_files(TARGET_FOLDER):
    #/home/jovyan/work/shared_downloads/Purpose/[今日の日付]/ のフォルダをなかったら作成、あってもif文抜けるだけ
    if not os.path.exists(TARGET_FOLDER):
        os.makedirs(TARGET_FOLDER)
    
    print("📂 ファイル移動開始...")
    for subject, courses in PURPOSE_CSV_URLS.items():
        for course_name, urls in courses.items():
            # 各ユーザーごとの移動先フォルダを作成
            target_dir = os.path.join(TARGET_FOLDER, subject, course_name)
            os.makedirs(target_dir, exist_ok=True)
    
            for filename in os.listdir(JUPYTER_SHARED_DIR):
                file_path = os.path.join(JUPYTER_SHARED_DIR, filename)
    
                # 対象がファイルかつ CSV のみ処理
                if os.path.isfile(file_path) and filename.lower().endswith(".csv"):
                    # ファイル名にsubjectとcourse名が両方含まれる場合振り分けてファイル移動する
                    if subject.lower() in filename.lower() and course_name.lower() in filename.lower():
                        target_path = os.path.join(target_dir, filename)
                        shutil.move(file_path, target_path)
                        print(f"✅ {filename} を {target_dir} に移動")
                    

def main():
    driver = setup_driver()
    try:
        print("hopeにログインしています")
        login_to_hope(driver)
        
        print("小テストのcsvのダウンロードを行います")
        minitest_download_csv_from_urls(driver, TEST_CSV_URLS, 'id')
        minitest_move_downloaded_files()
        
        # print("目標設定のcsvのダウンロードを行います")
        # time.sleep(60)
        # comment_purpose_download_csv_from_urls(driver, PURPOSE_CSV_URLS)
        # comment_purpose_move_downloaded_files(PURPOSE_TARGET_FOLDER)
        
        # print("目標設定のコメントのcsvのダウンロードを行います")
        # time.sleep(60)
        # comment_purpose_download_csv_from_urls(driver, COMMENT_PURPOSE_CSV_URLS)
        # comment_purpose_move_downloaded_files(COMMENT_TARGET_FOLDER)  
        
        print("🎉 全ダウンロード完了")
    finally:
        driver.quit()

if __name__ == "__main__":
    main()


hopeにログインしています
⌛ 「未来大の学生・教職員」リンクをクリック
⌛ ユーザー名・パスワード入力待機中
✅ ログイン成功
小テストのcsvのダウンロードを行います
🌐 アクセス中: https://hope.fun.ac.jp/grade/export/txt/index.php?id=2059
⬇️ ダウンロード中...（3秒待機）
🌐 アクセス中: https://hope.fun.ac.jp/grade/export/txt/index.php?id=2409
⬇️ ダウンロード中...（3秒待機）
🌐 アクセス中: https://hope.fun.ac.jp/grade/export/txt/index.php?id=2412
⬇️ ダウンロード中...（3秒待機）
📂 ファイル移動開始...
✅ keilabtest5 評定-20250525_0800-comma_separated.csv を /home/jovyan/work/shared_downloads/MiniTest/20250525 に移動
✅ keitest4 評定-20250525_0800-comma_separated.csv を /home/jovyan/work/shared_downloads/MiniTest/20250525 に移動
✅ keilabtest7 評定-20250525_0800-comma_separated.csv を /home/jovyan/work/shared_downloads/MiniTest/20250525 に移動
🎉 全ダウンロード完了
